In [1]:
# PLANILHA COM TODOS OS LAYERS DO TEMPLATE

# # Lista para armazenar informações sobre as layers
# layers_info = []

# # Iterar sobre todas as layers e coletar informações
# for layer in doc.layers:
#     layers_info.append({'Layer Name': layer.dxf.name})

# # Criar um DataFrame do pandas com as informações das layers
# df_layers = pd.DataFrame(layers_info)

# # Salvar o DataFrame em um arquivo Excel
# df_layers.to_excel('layers_info.xlsx', index=False, engine='openpyxl')

# print("Layers foram salvas em 'layers_info.xlsx'.")

In [22]:
# PLANILHA COM OS LAYERS EXISTENTES NO ARQUIVO

# def gerar_planilha_layers_tipos(layers_existentes, output_path):
#     # Abrir o arquivo DXF
#     doc = ezdxf.readfile(layers_existentes)
    
#     # Dicionário para armazenar layers e seus tipos
#     layers_e_tipos = {}

#     # Iterar sobre as entidades do modelo
#     for entity in doc.modelspace():
#         layer_name = entity.dxf.layer
#         entity_type = entity.dxftype()

#         # Adicionar o tipo ao dicionário se a layer ainda não foi registrada
#         if layer_name not in layers_e_tipos:
#             layers_e_tipos[layer_name] = entity_type

#     # Converter o dicionário para um DataFrame do pandas
#     df = pd.DataFrame(list(layers_e_tipos.items()), columns=['Layer', 'Tipo'])

#     # Salvar o DataFrame em uma planilha Excel
#     df.to_excel(output_path, index=False)

# # Exemplo de uso
# layers_existentes = r'C:\Users\matheus.rezende\OneDrive - Cocal\Projetos\Mapa_Sistematizacao\Dados\3556 - SISTEMATIZAÇÃO.dxf'
# output_path = r'C:\Users\matheus.rezende\OneDrive - Cocal\Projetos\Mapa_Sistematizacao\Documentos\layers_existentes1.xlsx'
# gerar_planilha_layers_tipos(layers_existentes, output_path)

In [65]:
# Dicionários para classificar layers
layers_ha = [
    '801 - Talhões', '802 - Caixa Seca-Caixa d\'água', '995 - Canal Escoadouro', '818 - Linha Transmissão_Alta Tensão',
    '815 - Sem Cana', '811 - Mato', '810 - App-Brejo-Varzea', '816 - Pedra', '819 - Linha Distribuição_Rede Eletrica',
    '996 - Canal de Vinhaça', '813 - Árvores_Reboleira', '817 - Sede-Mangueira', '803 - Encanamento', '821 - Perímetro-GEO',
    '812 - Reserva Legal', '814 - Eucalipto', '994 - Pátios', '998 - Área a preparar', '799 - Fornecedores', '798 - Compra',
    '993 - Sítio Arqueologico', '822 - Drenagem_Rios', '823 - Massas D\'água_Lagos - Represas', '824 - Divisa Empresa',
    '825 - Áreas Urbanas', '826 - Usinas', '828 - Perimetro', '830 - Pista de Pouso', '992 - Reflorestamento',
    '831 - Oficina', '827 - Perímetro Cadastro', 'TALHÕES', '807 - Linha Férrea', '820 - Cabo Ótico', '832 - Divisa Produção',
    '797 - Viveiro', '796 - Adutora', '765 - Estrutura Vinhaça', '764 - Torre Telefonia', '0 - NÃO Reforma', '834 - Área inaproveitável'
]

layers_metros_lineares = [
    '804 - Cerca', '809 - Estradas Pavimentadas', '806 - Eixo do Carreador', '805 - Curvas', '808 - Estradas não Pavimentadas',
    '0 - Carreador EXISTENTE', '0 - Carreador NOVO', '0 - Terraço EXISTENTE', '0 - Carreador à ELIMINAR', '0 - Terraço à ELIMINAR',
    '0 - Terraço NOVO', '3.0', 'SULCOS MESTRES', 'SULCOS MATAÇÃO', 'MEIOSI', 'DESDOBRA', '0 - Terraço EMB - BL',
    '0 - Cerca - Manter', '0 - Cerca - Contruir', '0 - Cerca - Eliminar', '0 - Cerca - Reformar'
]

layers_unidade = [
    '829 - Ponto de Monitoramento Ambiental', '0 - Levante - Existente', '0 - Unidade Desaguadouros', '0 - Levante - Canal',
    '0 - Desaguadouro - Canal', '0- Desaguadouros à Eliminar', '0 - Levante - Eliminar', '0 - Levante - Construir',
    '0 - Desaguadouro - Eliminar', '0 - Desaguadouro - Construir', '0 - Desaguadouro - Existente', '0 - Levante'
]

In [2]:
import ezdxf
from shapely.geometry import LineString, Polygon
import pandas as pd
import logging
import streamlit as st

# Configurar o logger para o ezdxf
logging.getLogger('ezdxf').setLevel(logging.ERROR)

In [78]:
def calcular_metros_lineares_e_contar_circulos_por_layer(dxf_path, layer_names_excluidos):
    # Abrir o arquivo DXF
    doc = ezdxf.readfile(dxf_path)
    
    # Dicionários para armazenar metros lineares e contagem de círculos por layer
    metros_por_layer = {}
    contagem_circulos_por_layer = {}

    # Iterar sobre as entidades do modelo
    for entity in doc.modelspace():
        layer_name = entity.dxf.layer

        if entity.dxftype() == 'LINE':
            # Verifica se a layer não está na lista de exclusão
            if layer_name not in layer_names_excluidos:
                # Cria uma linha com os pontos de início e fim
                line = LineString([entity.dxf.start, entity.dxf.end])
                length_in_meters = line.length

                # Acumula o comprimento no layer correspondente
                if layer_name in metros_por_layer:
                    metros_por_layer[layer_name] += length_in_meters
                else:
                    metros_por_layer[layer_name] = length_in_meters

        elif entity.dxftype() in {'LWPOLYLINE', 'POLYLINE'}:
            # Verifica se a layer não está na lista de exclusão
            if layer_name not in layer_names_excluidos:
                # Extrai os pontos da polilinha
                if entity.dxftype() == 'LWPOLYLINE':
                    points = entity.get_points('xy')
                else:  # POLYLINE
                    points = [point for point in entity.points()]
                
                line = LineString(points)
                length_in_meters = line.length

                # Acumula o comprimento no layer correspondente
                if layer_name in metros_por_layer:
                    metros_por_layer[layer_name] += length_in_meters
                else:
                    metros_por_layer[layer_name] = length_in_meters

        elif entity.dxftype() == 'CIRCLE':
            # Verifica se a layer não está na lista de exclusão
            if layer_name not in layer_names_excluidos:
                # Conta o número de círculos por layer
                if layer_name in contagem_circulos_por_layer:
                    contagem_circulos_por_layer[layer_name] += 1
                else:
                    contagem_circulos_por_layer[layer_name] = 1

        else:
            continue  # Ignorar outras entidades

    return metros_por_layer, contagem_circulos_por_layer

def contar_lwpolylines(dxf_path, layer_name):
    # Abrir o arquivo DXF
    doc = ezdxf.readfile(dxf_path)
    
    # Contador para a quantidade de LWPOLYLINE na layer especificada
    contador = 0

    # Iterar sobre as entidades do modelo
    for entity in doc.modelspace():
        if entity.dxftype() == 'LWPOLYLINE':
            if entity.dxf.layer == layer_name:
                contador += 1

    return contador

def calcular_area_total_em_hectares_por_layers(dxf_path, layer_names):
    # Abrir o arquivo DXF
    doc = ezdxf.readfile(dxf_path)
    
    # Dicionário para armazenar a área total por layer
    areas_por_layer = {layer: 0.0 for layer in layer_names}

    # Iterar sobre as entidades do modelo
    for entity in doc.modelspace():
        layer_name = entity.dxf.layer
        if layer_name in areas_por_layer:
            if entity.dxftype() in {'LWPOLYLINE', 'POLYLINE'}:
                # Extrai os pontos da polilinha
                if entity.dxftype() == 'LWPOLYLINE':
                    points = entity.get_points('xy')
                else:  # POLYLINE
                    points = [point for point in entity.points()]
                
                # Criar um polígono com os pontos
                polygon = Polygon(points)
                area_m2 = polygon.area
                areas_por_layer[layer_name] += area_m2

    # Converter as áreas de metros quadrados para hectares
    areas_por_layer_hectares = {layer: area / 10000.0 for layer, area in areas_por_layer.items()}

    return areas_por_layer_hectares

def calcular_area_diferenca(dxf_path, layer_total, layers_subtracao):
    # Calcular a área total da layer principal
    areas_totais_hectares = calcular_area_total_em_hectares_por_layers(dxf_path, [layer_total])
    area_total_hectares = areas_totais_hectares.get(layer_total, 0.0)

    # Calcular a soma das áreas das layers a serem subtraídas
    areas_subtracao_hectares = calcular_area_total_em_hectares_por_layers(dxf_path, layers_subtracao)
    area_subtracao_hectares = sum(areas_subtracao_hectares.values())

    # Calcular a área dos carreadores
    area_carreadores_hectares = area_total_hectares - area_subtracao_hectares

    return area_carreadores_hectares

# Exemplo de uso
dxf_path = r'C:\Users\matheus.rezende\OneDrive - Cocal\Projetos\Mapa_Sistematizacao\Dados\3611 - SISTEMATIZAÇÃO PARTE 1-2.dxf'
layer_names_excluidos = ['827 - Perímetro Cadastro', 'Não Reforma', "818 - Linha Transmissão_Alta Tensão", "0 -Coroamento Postes",
                         "819 - Linha Distribuição_Rede Eletrica", "815 - Sem Cana", "TALHÕES", "ÁREAS", "Texto", "801 - Talhões", "0"]

metros_por_layer, contagem_circulos_por_layer = calcular_metros_lineares_e_contar_circulos_por_layer(dxf_path, layer_names_excluidos)

print("Metros Lineares:")
for layer, metros in metros_por_layer.items():
    print(f"Layer: {layer}: {metros:.2f} m")

print("\nContagem de Unidades:")
for layer, count in contagem_circulos_por_layer.items():
    print(f"Layer: {layer}: {count} unidades")

layer_name = '0 -Coroamento Postes'
quantidade = contar_lwpolylines(dxf_path, layer_name)
print(f"Layer: {layer_name}: {quantidade} unidades")

layer_names = ['827 - Perímetro Cadastro', 'Não Reforma', "818 - Linha Transmissão_Alta Tensão",
               "819 - Linha Distribuição_Rede Eletrica", "815 - Sem Cana", "TALHÕES"]
areas_totais_hectares = calcular_area_total_em_hectares_por_layers(dxf_path, layer_names)

print("\nÁreas Totais em Hectares:")
for layer, area in areas_totais_hectares.items():
    print(f"Layer: {layer}: {area:.2f} ha")

layer_total = '827 - Perímetro Cadastro'
layers_subtracao = ['Não Reforma', "818 - Linha Transmissão_Alta Tensão",
                    "819 - Linha Distribuição_Rede Eletrica", "815 - Sem Cana", "TALHÕES"]

area_carreadores = calcular_area_diferenca(dxf_path, layer_total, layers_subtracao)
print(f"Layer: Área dos Carreadores: {area_carreadores:.2f} ha")

Metros Lineares:
Layer: 0-BASE LARGA - ELIMINAR: 1578.66 m
Layer: 0 - Terraço EMB - BL: 7978.13 m
Layer: 0 - Carreador EXISTENTE: 12746.82 m
Layer: 0-EMBUTIDA - ELIMINAR: 989.25 m
Layer: Terraço Embutido: 781.27 m
Layer: 0 - Carreador NOVO: 2359.61 m
Layer: 0 - Terraço NOVO: 35.07 m

Contagem de Unidades:
Layer: 0 - Levante - Existente: 90 unidades
Layer: 0 - Desaguadouro - Existente: 77 unidades
Layer: 0 - Levante - Construir: 22 unidades
Layer: 0 - Desaguadouro - Construir: 23 unidades
Layer: 0 -Coroamento Postes: 20 unidades

Áreas Totais em Hectares:
Layer: 827 - Perímetro Cadastro: 262.39 ha
Layer: Não Reforma: 143.55 ha
Layer: 818 - Linha Transmissão_Alta Tensão: 4.13 ha
Layer: 819 - Linha Distribuição_Rede Eletrica: 0.09 ha
Layer: 815 - Sem Cana: 2.23 ha
Layer: TALHÕES: 106.33 ha
Layer: Área dos Carreadores: 6.05 ha


In [79]:
# Criar DataFrames individuais com a unidade de medida
df_metros = pd.DataFrame({
    'Layer': list(metros_por_layer.keys()),
    'Número': [round(value, 2) for value in metros_por_layer.values()],
    'Unidade de Medida': 'metros lineares'
})

df_circulos = pd.DataFrame({
    'Layer': list(contagem_circulos_por_layer.keys()),
    'Número': list(contagem_circulos_por_layer.values()),
    'Unidade de Medida': 'unidades'
})

df_areas = pd.DataFrame({
    'Layer': list(areas_totais_hectares.keys()),
    'Número': [round(value, 2) for value in areas_totais_hectares.values()],
    'Unidade de Medida': 'ha'
})

df_carreadores = pd.DataFrame({
    'Layer': ['Carreador Estimado'],
    'Número': [round(area_carreadores, 2)],
    'Unidade de Medida': 'ha'
})

# Criar uma planilha com todas as tabelas na mesma aba
with pd.ExcelWriter(r'C:\Users\matheus.rezende\OneDrive - Cocal\Projetos\Mapa_Sistematizacao\Dados\teste.xlsx', engine='openpyxl') as writer:
    start_col = 0

    # Escrever DataFrame de metros lineares
    df_metros.to_excel(writer, sheet_name='Resumo', startrow=0, startcol=start_col, index=False)
    start_col += len(df_metros.columns) + 1  # Deixar espaço entre tabelas

    # Escrever DataFrame de círculos
    df_circulos.to_excel(writer, sheet_name='Resumo', startrow=0, startcol=start_col, index=False)
    start_col += len(df_circulos.columns) + 1  # Deixar espaço entre tabelas

    # # Escrever DataFrame de áreas
    # df_areas.to_excel(writer, sheet_name='Resumo', startrow=0, startcol=start_col, index=False)
    # start_col += len(df_areas.columns) + 1  # Deixar espaço entre tabelas

    # # Escrever DataFrame de carreadores
    # df_carreadores.to_excel(writer, sheet_name='Resumo', startrow=0, startcol=start_col, index=False)

    # Concatenar os DataFrames verticalmente (uma tabela abaixo da outra)
    df_combined = pd.concat([df_areas, df_carreadores], ignore_index=True)

    df_combined.to_excel(writer, sheet_name='Resumo', startrow=0, startcol=start_col, index=False)

    # Obter o workbook e a worksheet
    # workbook  = writer.book
    # worksheet = writer.sheets['Resumo']

print("Planilha criada com sucesso.")

Planilha criada com sucesso.
